In [1]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate()

  Activating environment at `~/Git/tinympc-julia/state_constraints/Project.toml`


In [2]:
using RobotDynamics
using ForwardDiff
using TrajOptPlots
using BlockDiagonals
using LinearAlgebra
using StaticArrays
using SparseArrays

using Plots
using Printf

include("../quaternion-stuff.jl")

E (generic function with 1 method)

In [35]:
#Fixed-Point Conversion
r_max = 100.0
q_max = 10.0
ϕ_max = 30.0
v_max = 100.0
ω_max = 200.0
control_max = 0.5*m*gravity

# fp_max = 2147483647.0
fp_max = 2147483647.0

J_xfp = Diagonal([ones(3)*(r_max); ones(3)*(ϕ_max); ones(3)*(v_max); ones(3)*(ω_max)])
J_ufp = Diagonal(ones(4)*(control_max))

J_xfp_inv = J_xfp\I
J_ufp_inv = J_ufp\I

function state_float_to_fixed(x)
    x_fixed = zeros(Int32, 13)
    x_fixed[1:3] .= convert(Vector{Int32},round.(x[1:3].*(fp_max/r_max)))
    x_fixed[4:7] .= convert(Vector{Int32},round.(x[4:7].*(fp_max/q_max)))
    x_fixed[8:10] .= convert(Vector{Int32},round.(x[8:10].*(fp_max/v_max)))
    x_fixed[11:13] .= convert(Vector{Int32},round.(x[11:13].*(fp_max/ω_max)))
    
    return x_fixed
end

function Δstate_float_to_fixed(x)
    x_fixed = zeros(Int32, 12)
    x_fixed[1:3] .= convert(Vector{Int32},round.(x[1:3].*(fp_max/r_max)))
    x_fixed[4:6] .= convert(Vector{Int32},round.(x[4:6].*(fp_max/ϕ_max)))
    x_fixed[7:9] .= convert(Vector{Int32},round.(x[7:9].*(fp_max/v_max)))
    x_fixed[10:12] .= convert(Vector{Int32},round.(x[10:12].*(fp_max/ω_max)))
    
    return x_fixed
end

function state_fixed_to_float(x)
    x_float = zeros(13)
    x_float[1:3] .= convert(Vector{Float64},x[1:3]).*(r_max/fp_max)
    x_float[4:7] .= convert(Vector{Float64},x[4:7]).*(q_max/fp_max)
    x_float[8:10] .= convert(Vector{Float64},x[8:10]).*(v_max/fp_max)
    x_float[11:13] .= convert(Vector{Float64},x[11:13]).*(ω_max/fp_max)
    
    return x_float
end

function Δstate_fixed_to_float(x)
    x_float = zeros(12)
    x_float[1:3] .= convert(Vector{Float64},x[1:3]).*(r_max/fp_max)
    x_float[4:6] .= convert(Vector{Float64},x[4:6]).*(ϕ_max/fp_max)
    x_float[7:9] .= convert(Vector{Float64},x[7:9]).*(v_max/fp_max)
    x_float[10:12] .= convert(Vector{Float64},x[10:12]).*(ω_max/fp_max)
    
    return x_float
end

function controls_float_to_fixed(u)
    u_fixed = zeros(Int32, 4)
    u_fixed[1:4] .= convert(Vector{Int32},round.((u[1:4]).*(fp_max/control_max)))
    # u_fixed[1:4] .= convert(Vector{Int32},(u[1:4]).*(fp_max/control_max))
    
    return u_fixed
end

function controls_fixed_to_float(u)
    u_float = zeros(4)
    u_float[1:4] .= convert(Vector{Float64}, u).*(control_max/fp_max)
    
    return u_float
end

controls_fixed_to_float (generic function with 1 method)

In [22]:
function quad_dynamics(x,u)
    r = x[1:3]
    q = x[4:7]/norm(x[4:7]) #normalize q just to be careful
    v = x[8:10]
    ω = x[11:13]
    Q = qtoQ(q)
    
    ṙ = v
    q̇ = 0.5*L(q)*H*ω
    
    b = 1/m
  
    v̇ = [0; 0; -gravity] + (1/m)*Q*[zeros(2,4); kt*ones(1,4)]*u 
  
    ω̇ = J\(-hat(ω)*J*ω + [-ℓ*kt -ℓ*kt ℓ*kt ℓ*kt; -ℓ*kt ℓ*kt ℓ*kt -ℓ*kt; -km km -km km]*u)
    
    return [ṙ; q̇; v̇; ω̇]
  end
  function quad_dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5*h*f1, u)
    f3 = quad_dynamics(x + 0.5*h*f2, u)
    f4 = quad_dynamics(x + h*f3, u)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    xn[4:7] = xn[4:7]/norm(xn[4:7]) #re-normalize quaternion
    return xn
  end

quad_dynamics_rk4 (generic function with 1 method)

In [49]:
# mass_scale = 1/.5
# length_scale = 1/.1750
# time_scale = 1/.05

mass_scale = 1/.5
length_scale = 1/.1750
time_scale = 1

#Quadrotor parameters
m = 0.5 * mass_scale # kg
ℓ = 0.1750 * length_scale # meters
J = Diagonal([0.0023, 0.0023, 0.004]) * mass_scale * length_scale^2 # kg * meters^2
gravity = 9.81 * length_scale / (time_scale^2)
kt = 1.0 * mass_scale * length_scale / (time_scale^2) # N
km = 0.0245 * mass_scale * length_scale^2 / (time_scale^2) # Nm

h = 0.05 # 20 Hz

Nx = 13     # number of states (quaternion)
Nx̃ = 12     # number of states (linearized)
Nu = 4     # number of controls
nx = Nx̃
nu = Nu

# Goal state
@show uhover = (m*gravity/kt/4)*ones(4) # m = 30g and max thrust = 60g
# @show uhover = (m*gravity/4)*ones(4) # m = 30g and max thrust = 60g
rg = [0.0; 0; 0.0]
qg = [1.0; 0; 0; 0]
vg = zeros(3)
ωg = zeros(3)
xg = [rg; qg; vg; ωg];

uhover = (((m * gravity) / kt) / 4) * ones(4) = [1.22625, 1.22625, 1.22625, 1.22625]


In [50]:
#Linearize dynamics about hover
Adyn = ForwardDiff.jacobian(x->quad_dynamics_rk4(x,uhover),xg)
Bdyn = ForwardDiff.jacobian(u->quad_dynamics_rk4(xg,u),uhover);
Ãdyn = Array(E(qg)'*Adyn*E(qg))
B̃dyn = Array(E(qg)'*Bdyn);

In [53]:
Tfinal = 5.0            # final time
N = Int(Tfinal/h)+1     # number of time steps
t_vec = h*(0:N-1)

# Cost weights
# Q = Array(BlockDiagonal([5.0*I(3), 0.001*I(Nx̃-3)]));
# R = Array(1*I(Nu));
# Qf = 1*Q;

# Cost weights
max_dev_x = [0.0316; 0.0316; 0.0316;  0.5; 0.5; 0.5;  0.5; 0.5; 0.5;  0.7; 0.7; 0.5]
# max_dev_u = [0.5; 0.5; 0.5; 0.5]
max_dev_u = [1; 1; 1; 1]
Q = diagm(1 ./(max_dev_x.^2))
R = diagm(1 ./(max_dev_u.^2))
Qf = 1*Q

# # Cost weights
Q = Array(I(Nx̃));
R = Array(I(Nu));
Qf = 1*Q

# Penalty
ρ = 5.0
R̃ = R + ρ*I;
# R̃ = R

Nriccati = 10000
P = [zeros(nx,nx) for i = 1:Nriccati];   # cost to go quadratic term
K = [zeros(nu,nx) for i = 1:Nriccati-1]; # feedback gain

P[Nriccati] .= Qf;
for k = (Nriccati-1):-1:1
    K[k] = (R̃ + B̃dyn'*P[k+1]*B̃dyn)\(B̃dyn'*P[k+1]*Ãdyn);
    P[k] = Q + Ãdyn'*P[k+1]*(Ãdyn - B̃dyn*K[k]);
end

# Fixed point versions before rounding
Ã_fixed_preround = J_xfp_inv*Ãdyn*J_xfp
B̃_fixed_preround = J_xfp_inv*B̃dyn*J_ufp
Kinf_fixed_preround = J_ufp_inv*K[1]*J_xfp
Pinf_fixed_preround = J_xfp_inv*P[1]*J_xfp

Qu1_fixed_preround = J_ufp_inv*(R̃ + B̃dyn'*P[1]*B̃dyn)^(-1)*B̃dyn'*J_xfp
Qu2_fixed_preround = J_ufp_inv*(R̃ + B̃dyn'*P[1]*B̃dyn)^(-1)*J_ufp

AmBKt_fixed_preround = J_xfp_inv*(Ãdyn - B̃dyn*K[1])'*J_xfp
coeff_d2p_fixed_preround = J_xfp_inv*(K[1]'*R̃ - (Ãdyn - B̃dyn*K[1])'*P[1]*B̃dyn)*J_ufp
Kt_fixed_preround = J_xfp_inv*K[1]'*J_ufp

ρ_fixed_preround = J_ufp_inv*ρ*J_ufp

Q_fixed_preround = J_xfp_inv*Q*J_xfp
R_fixed_preround = J_ufp_inv*R*J_ufp
Qf_fixed_preround = J_xfp_inv*Qf*J_xfp

# Round fixed point versions of floating point nums
Ã_fixed = round.(Ã_fixed_preround)
B̃_fixed = round.(B̃_fixed_preround)
Kinf_fixed = round.(Kinf_fixed_preround)
Pinf_fixed = round.(Pinf_fixed_preround)

Qu1_fixed = round.(Qu1_fixed_preround)
Qu2_fixed = round.(Qu2_fixed_preround)

AmBKt_fixed = round.(AmBKt_fixed_preround)
coeff_d2p_fixed = round.(coeff_d2p_fixed_preround)
Kt_fixed = round.(Kt_fixed_preround)

ρ_fixed = round.(ρ_fixed_preround)

Q_fixed = round.(Q_fixed_preround)
R_fixed = round.(R_fixed_preround)
Qf_fixed = round.(Qf_fixed_preround)
;

In [54]:
display("Ã")
display(Ã_fixed_preround)
display(rank(Ã_fixed_preround))
display(Ã_fixed)
display(rank(Ã_fixed))

display("B̃")
display(B̃_fixed_preround)
display(rank(B̃_fixed_preround))
display(B̃_fixed)
display(rank(B̃_fixed))

display("Kinf")
display(Kinf_fixed_preround)
display(rank(Kinf_fixed_preround))
display(Kinf_fixed)
display(rank(Kinf_fixed))

display("Pinf")
display(Pinf_fixed_preround)
display(rank(Pinf_fixed_preround))
display(Pinf_fixed)
display(rank(Pinf_fixed))

display("Qu1")
display(Qu1_fixed_preround)
display(rank(Qu1_fixed_preround))
display(Qu1_fixed)
display(rank(Qu1_fixed))

display("Qu2")
display(Qu2_fixed_preround)
display(rank(Qu2_fixed_preround))
display(Qu2_fixed)
display(rank(Qu2_fixed))

display("AmBKt")
display(AmBKt_fixed_preround)
display(rank(AmBKt_fixed_preround))
display(AmBKt_fixed)
display(rank(AmBKt_fixed))

display("coeff_d2p")
display(coeff_d2p_fixed_preround)
display(rank(coeff_d2p_fixed_preround))
display(coeff_d2p_fixed)
display(rank(coeff_d2p_fixed))

display("Kt")
display(Kt_fixed_preround)
display(rank(Kt_fixed_preround))
display(Kt_fixed)
display(rank(Kt_fixed))

display("ρ")
display(ρ_fixed_preround)
display(rank(ρ_fixed_preround))
display(ρ_fixed)
display(rank(ρ_fixed))

display("Q")
display(Q_fixed_preround)
display(rank(Q_fixed_preround))
display(Q_fixed)
display(rank(Q_fixed))

display("R")
display(R_fixed_preround)
display(rank(R_fixed_preround))
display(R_fixed)
display(rank(R_fixed))

display("Qf")
display(Qf_fixed_preround)
display(rank(Qf_fixed_preround))
display(Qf_fixed)
display(rank(Qf_fixed))

"Ã"

12×12 Matrix{Float64}:
 1.0  0.0  0.0   0.0        0.0420429  …   0.0         0.00233571  0.0
 0.0  1.0  0.0  -0.0420429  0.0           -0.00233571  0.0         0.0
 0.0  0.0  1.0   0.0        0.0            0.0         0.0         0.0
 0.0  0.0  0.0   1.0        0.0            0.166667    0.0         0.0
 0.0  0.0  0.0   0.0        1.0            0.0         0.166667    0.0
 0.0  0.0  0.0   0.0        0.0        …   0.0         0.0         0.166667
 0.0  0.0  0.0   0.0        1.68171        0.0         0.140143    0.0
 0.0  0.0  0.0  -1.68171    0.0           -0.140143    0.0         0.0
 0.0  0.0  0.0   0.0        0.0            0.0         0.0         0.0
 0.0  0.0  0.0   0.0        0.0            1.0         0.0         0.0
 0.0  0.0  0.0   0.0        0.0        …   0.0         1.0         0.0
 0.0  0.0  0.0   0.0        0.0            0.0         0.0         1.0

12

12×12 Matrix{Float64}:
 1.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
 0.0  1.0  0.0  -0.0  0.0  0.0  0.0  0.0  0.0  -0.0  0.0  0.0
 0.0  0.0  1.0   0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
 0.0  0.0  0.0   1.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
 0.0  0.0  0.0   0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
 0.0  0.0  0.0   0.0  0.0  1.0  0.0  0.0  0.0   0.0  0.0  0.0
 0.0  0.0  0.0   0.0  2.0  0.0  1.0  0.0  0.0   0.0  0.0  0.0
 0.0  0.0  0.0  -2.0  0.0  0.0  0.0  1.0  0.0  -0.0  0.0  0.0
 0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  1.0   0.0  0.0  0.0
 0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0   1.0  0.0  0.0
 0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0   0.0  1.0  0.0
 0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  1.0

12

...

"Qf"

12×12 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

12

12×12 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

12